In [11]:
!pip install torchvision

  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 41.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   --- ------------------------------------ 17.3/204.1 MB 83.9 MB/s eta 0:00:03
   ------- -------------------------------- 35.9/204.1 MB 87.8 MB/s eta 0:00:02
   ---------- ----------------------------- 51.1/204.1 MB 81.4 MB/s eta 0:00:02
   ------------- -------------------------- 70.3/204.1 MB 84.5 MB/s eta 0:00:02
   ----------------- ---------------------- 89.4/204.1 MB 85.1 MB/s eta 0:00:02
   -------------------- ------------------ 108.5/204.1 MB 85.5 MB/s eta 0:00:02
   ------------------------ -------------- 126.4/204.1 MB 85.0 MB/s eta 0:00:01
   --------------------------- ----------- 145.2/204.1 MB 85.1 MB/s eta 0:00:01
   ------------------------------- ------- 163.8/204.1 MB 85.1 MB/s eta 0:00:01
 

In [12]:
!pip install qiskit-algorithms

In [41]:
!pip install qiskit-optimization

     ---------------------------------------- 0.0/645.5 kB ? eta -:--:--
     ------------------------------------- 645.5/645.5 kB 24.0 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for docplex: filename=docplex-2.29.241-py3-none-any.whl size=685474 sha256=8ca174373f632b31f31dbf540ac492fce96004ec47fa99f069de4e6e72b9d36a
  Stored in directory: c:\users\jovin\appdata\local\pip\cache\wheels\1e\6d\85\f2bcad82728552de08dfcaec5dd4dfd2dd75284a29312e95c5
Successfully built docplex


In [27]:
print(qiskit.__version__)

2.0.0


In [42]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, transpile
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.circuit.library import RealAmplitudes
from qiskit_algorithms.optimizers import COBYLA

ImportError: cannot import name 'BaseSampler' from 'qiskit.primitives' (C:\Users\jovin\anaconda3\envs\ECE570\lib\site-packages\qiskit\primitives\__init__.py)

In [56]:
import qiskit
from qiskit.circuit import QuantumCircuit, QuantumRegister, ParameterVector
from qiskit.quantum_info import Statevector
import numpy as np
from scipy.signal import stft  # Example for classical preprocessing
from qiskit_aer import Aer

In [57]:
def preprocess_audio(audio_file):
    sampling_rate = 44100
    duration = 2  # seconds
    num_frames = 10
    time_points = np.linspace(0, duration, num_frames, endpoint=False)
    features = np.random.rand(num_frames, 4)  
    return features, time_points

In [58]:
def create_variational_circuit(num_qubits, num_features, num_parameters):
    qr = QuantumRegister(num_qubits, 'q')
    qc = QuantumCircuit(qr, name='var_circ')
    params = ParameterVector('theta', num_parameters)

    for i in range(min(num_features, num_qubits)):
        qc.ry(params[i], qr[i])

    if num_qubits > 1 and num_parameters > min(num_features, num_qubits):
        qc.cx(qr[0], qr[1])
        qc.rz(params[min(num_features, num_qubits)], qr[1]) 
        qc.cx(qr[0], qr[1])

    return qc, params

In [59]:
def encode_and_run(features, time_point, variational_circuit, parameters, backend):
    num_features_local = features.shape[0]
    num_qubits = variational_circuit.num_qubits
    num_params = len(parameters)

    param_values = features.tolist() + [np.random.rand()] * (num_params - num_features_local)

    bound_circuit = variational_circuit.assign_parameters(dict(zip(parameters, param_values)))

    # For a simulator:
    simulator = Aer.get_backend('aer_simulator')
    compiled_circuit = qiskit.transpile(bound_circuit, simulator)
    job = simulator.run(compiled_circuit, shots=1024)
    result = job.result()
    counts = result.get_counts(bound_circuit)

    predicted_note = interpret_counts(counts)

    return predicted_note, time_point

In [60]:
def interpret_counts(counts):

    most_likely_bitstring = max(counts, key=counts.get)

    note_mapping = {"00": "C4", "01": "D4", "10": "E4", "11": "F4"}
    return note_mapping.get(most_likely_bitstring, "Unknown")

In [55]:
if __name__ == "__main__":
    audio_file = "archive\Piano_Notes\piano-bb_A#_major.wav"  
    extracted_features, time_stamps = preprocess_audio(audio_file)

    num_qubits = 2  # Example
    num_features_per_frame = extracted_features.shape[1]
    num_variational_params = num_features_per_frame + 1 # Example: Allows for parameters beyond the input features

    variational_circuit, parameters = create_variational_circuit(num_qubits, num_features_per_frame, num_variational_params)

    # Choose a backend (simulator for now)
    backend = Aer.get_backend('aer_simulator')

    predicted_notes_with_time = []
    for i, features in enumerate(extracted_features):
        time_point = time_stamps[i]
        predicted_note, time = encode_and_run(features, time_point, variational_circuit, parameters, backend)
        predicted_notes_with_time.append((time, predicted_note))

    print("Predicted Notes with Time:")
    for time, note in predicted_notes_with_time:
        print(f"Time: {time:.2f}s, Note: {note}")

CircuitError: 'Cannot bind parameters (theta[3], theta[4]) not present in the circuit.'